In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
writer = tf.python_io.TFRecordWriter("../data/rnn_train.tfrecords")

In [3]:
train_X = np.load('../data/rnn_train.npy')
train_y = np.load('../data/rnn_label.npy')

In [4]:
train_X.shape

(1221232, 1470)

In [7]:
for idx in xrange(train_X.shape[0]):
    example = tf.train.Example(features=tf.train.Features(feature={
        "label": tf.train.Feature(float_list=tf.train.FloatList(value=[train_y[idx]])),
        "sequence": tf.train.Feature(float_list=tf.train.FloatList(value=[train_X[idx]]))
    }))
    writer.write(example.SerializeToString())
writer.close()

TypeError: only length-1 arrays can be converted to Python scalars

In [8]:
tf.train.Feature?

In [6]:
train_y[0]

-0.016528925

In [9]:
train_X[0]

array([-0.09090909, -0.08987603, -0.08884297, ...,  0.        ,
       -0.00309917,  0.        ], dtype=float32)

In [11]:
def np_to_tfrecords(X, Y, file_path_prefix, verbose=True):
    """
    Converts a Numpy array (or two Numpy arrays) into a tfrecord file.
    For supervised learning, feed training inputs to X and training labels to Y.
    For unsupervised learning, only feed training inputs to X, and feed None to Y.
    The length of the first dimensions of X and Y should be the number of samples.
    
    Parameters
    ----------
    X : numpy.ndarray of rank 2
        Numpy array for training inputs. Its dtype should be float32, float64, or int64.
        If X has a higher rank, it should be rshape before fed to this function.
    Y : numpy.ndarray of rank 2 or None
        Numpy array for training labels. Its dtype should be float32, float64, or int64.
        None if there is no label array.
    file_path_prefix : str
        The path and name of the resulting tfrecord file to be generated, without '.tfrecords'
    verbose : bool
        If true, progress is reported.
    
    Raises
    ------
    ValueError
        If input type is not float (64 or 32) or int.
    
    """
    def _dtype_feature(ndarray):
        """match appropriate tf.train.Feature class with dtype of ndarray. """
        assert isinstance(ndarray, np.ndarray)
        dtype_ = ndarray.dtype
        if dtype_ == np.float64 or dtype_ == np.float32:
            return lambda array: tf.train.Feature(float_list=tf.train.FloatList(value=array))
        elif dtype_ == np.int64:
            return lambda array: tf.train.Feature(int64_list=tf.train.Int64List(value=array))
        else:  
            raise ValueError("The input should be numpy ndarray. \
                               Instaed got {}".format(ndarray.dtype))
            
    assert isinstance(X, np.ndarray)
    assert len(X.shape) == 2  # If X has a higher rank, 
                               # it should be rshape before fed to this function.
    assert isinstance(Y, np.ndarray) or Y is None
    
    # load appropriate tf.train.Feature class depending on dtype
    dtype_feature_x = _dtype_feature(X)
    if Y is not None:
        assert X.shape[0] == Y.shape[0]
        assert len(Y.shape) == 2
        dtype_feature_y = _dtype_feature(Y)            
    
    # Generate tfrecord writer
    result_tf_file = file_path_prefix + '.tfrecords'
    writer = tf.python_io.TFRecordWriter(result_tf_file)
    if verbose:
        print "Serializing {:d} examples into {}".format(X.shape[0], result_tf_file)
        
    # iterate over each sample,
    # and serialize it as ProtoBuf.
    for idx in range(X.shape[0]):
        x = X[idx]
        if Y is not None:
            y = Y[idx]
        
        d_feature = {}
        d_feature['X'] = dtype_feature_x(x)
        if Y is not None:
            d_feature['Y'] = dtype_feature_y(y)
            
        features = tf.train.Features(feature=d_feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)
    
    if verbose:
        print "Writing {} done!".format(result_tf_file)


In [18]:
np_to_tfrecords(train_X, train_y, '../data/rnn_train')

Serializing 1221232 examples into ../data/rnn_train.tfrecords
Writing ../data/rnn_train.tfrecords done!


In [13]:
train_y.shape

(1221232,)

In [14]:
train_X.shape

(1221232, 1470)

In [17]:
train_y = train_y.reshape([1221232, 1])

In [3]:
filename_queue = tf.train.string_input_producer(['../data/rnn_train.tfrecords'], num_epochs=1)
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized_example,
    # Defaults are not specified since both keys are required.
    features={
        'X': tf.FixedLenFeature([], tf.float32),
        'Y': tf.FixedLenFeature([], tf.float32)
    })
train_f, train_l = tf.train.shuffle_batch([features['X'], features['Y']], batch_size=10, 
                                          capacity=128, num_threads=2, min_after_dequeue=10)


In [ ]:
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())


with tf.Session() as sess:
    sess.run(init_op)
    f, l = sess.run([train_f, train_l])
    print(f.shape)